In [1]:
import cx_Oracle

# --- Database Connection Setup ---
dsn = cx_Oracle.makedsn("localhost", 1522, service_name="XE")
conn = cx_Oracle.connect(user="system", password="system", dsn=dsn)
cursor = conn.cursor()

# --- Create Table (if not exists) ---
cursor.execute("""
BEGIN
   EXECUTE IMMEDIATE 'CREATE TABLE banks (
       bank_id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
       bank_name VARCHAR2(100) UNIQUE
   )';
EXCEPTION
   WHEN OTHERS THEN
      IF SQLCODE != -955 THEN -- ORA-00955: name is already used by an existing object
         RAISE;
      END IF;
END;
""")

# --- Sample Bank List ---
banks = [ "Dashen Bank", "CBE",  "Abyssinia Bank"]

# --- Insert Banks and Get IDs ---
bank_id_map = {}

for bank in banks:
    bank_id_var = cursor.var(int)
    try:
        cursor.execute(
            "INSERT INTO banks (bank_name) VALUES (:1) RETURNING bank_id INTO :2",
            [bank, bank_id_var]
        )
        bank_id_map[bank] = bank_id_var.getvalue()[0]
    except cx_Oracle.IntegrityError:
        # Bank already exists, fetch its ID
        cursor.execute("SELECT bank_id FROM banks WHERE bank_name = :1", [bank])
        bank_id_map[bank] = cursor.fetchone()[0]

# --- Commit changes ---
conn.commit()

# --- Print Result ---
print("Inserted or existing banks with IDs:")
for name, id in bank_id_map.items():
    print(f"{name}: {id}")

# --- Cleanup ---
cursor.close()
conn.close()


Inserted or existing banks with IDs:
Dashen Bank: 4
CBE: 5
Abyssinia Bank: 7
